Zostanie pokazane, jak można wdrożyć optymalizację symulowanego wyżarzania. 

Najpierw musimy zdefiniować naszą funkcję celu i granice kazdej zmiennej wejściowej do funkcji celu. 

Funkcja celu to tylko funkcja Pythona, którą będziemy nazywali  `objective()`. Granice będą tablicą 2D z jednym wymiarem dla każdej zmiennej wejściowej, który definiuje minimum i maksimum dla zmiennej. 

Na przykład jednowymiarowa funkcja celu i granice mozna zdefiniowac w nastepujacy sposób:

In [ ]:
# import niezbednych bibliotek
import numpy as np
from numpy.random import rand
from numpy import arange
from numpy.random import seed
from numpy import asarray
from numpy import exp
from numpy.random import randn

In [ ]:
list = ['NFT', 'NFTsMarketplace', 'NFTExchange', 'NFTgiveaway', 'NFTMarketplace', 'NFTmarket', 'NFTArt_Finance', 'OpenSeaNFT' ]

In [ ]:
def objective(x):
    return 0
# define range for input
bounds = np.asarray([[-5.0, 5.0]])

Następnie możemy wygenerować nasz punkt początkowy jako punkt losowy w granicach problemu,
kolejno można ocenic to za pomocą funkcji celu.

In [ ]:
# generate an initial point
best = bounds[:, 0] + rand(len(bounds)) * (bounds[:, 1] - bounds[:, 0])
# evaluate the initial point
best_eval = objective(best)

Musimy zachować „aktualne” rozwiązanie, na którym koncentruje się wyszukiwanie i które może być
zastąpione lepszymi rozwiązaniami.

In [ ]:
# current working solution
curr, curr_eval = best, best_eval

Teraz możemy zapętlić predefiniowaną liczbę iteracji algorytmu zdefiniowanego jako `n_iterations`, np. 100 lub 1000.

In [ ]:
for i in range(n_iterations):

Pierwszym krokiem iteracji algorytmu jest wygenerowanie nowego kandydata na rozwiązanie z obecnego
dzialajacego rozwiazania, np. zrob nastepny krok. Wymaga to wstępnie zdefiniowanego parametru `step_size`, który:
odnosi się do granic przestrzeni wyszukiwania. 

Zrobimy losowy krok z rozkladem Gaussa gdzie średnia jest naszym aktualnym punktem, a odchylenie standardowe jest określone przez
`“step_size`. 

Oznacza to, że około 99 procent podjętych kroków będzie mieściło się w 3 * step_size aktualnego punktu.

In [ ]:
candidate = solution + randn(len(bounds)) * step_siz

Nie musimy w ten sposób podejmować kroków. Możemy chcieć użyć równomiernego rozkładu między 0 a rozmiar kroku. 

Na przykład:

In [ ]:
candidate = solution + rand(len(bounds)) * step_size

Następnie musimy to ocenić.

In [ ]:
candidate_eval = objective(candidate)

Następnie musimy sprawdzić, czy ocena tego nowego punktu jest tak dobra lub lepsza niż aktualna
najlepszy punkt. Jeśli tak jest to zastąpić nasz obecny najlepszy punkt tym nowym punktem. 

To jest oddzielone od aktualnie dzialajacego rozwiazania, które jest przedmiotem poszukiwań.

In [ ]:
if candidate_eval < best_eval:
    # store new best point
    best, best_eval = candidate, candidate_eval
    # report progress
    print('>%d f(%s) = %.5f' % (i, best, best_eval))

Następnie musimy przygotować się do wymiany obecnego działającego rozwiązania. 

Pierwszym krokiem jest obliczenie różnicy między oceną funkcji celu obecnego rozwiązania a obecnym dzialajacym rozwiazaniem.


In [ ]:
diff = candidate_eval - curr_eval

Następnie musimy obliczyć aktualną temperaturę, korzystając z harmonogramu szybkiego wyżarzania, gdzie `temp` to początkowa temperatura podana jako argument.

In [ ]:
t = temp / float(i + 1)

Możemy wtedy obliczyć prawdopodobieństwo zaakceptowania rozwiązania o gorszej wydajności niż nasze obecne dzialajace rozwiazanie.

In [ ]:
metropolis = exp(-diff / t)

Wreszcie możemy zaakceptować nowy punkt jako obecnie dzialajace rozwiązanie, jeśli ma lepszą ocenę fukcji celu (różnica jest ujemna) lub jeśli funkcja celu jest gorsza, ale my prawdopodobnie zdecydujemy się to zaakceptować.

In [ ]:
if diff < 0 or rand() < metropolis:
    # store the new current point
    curr, curr_eval = candidate, candidate_eval

Możemy zaimplementować ten symulowany algorytm wyżarzania jako funkcję wielokrotnego użytku, która przyjmuje nazwę jako funkcję celu, granice każdej zmiennej wejściowej, suma iteracji, wielkość kroku i zwraca najlepsze znalezione rozwiązanie i jego ocenę.

In [ ]:
def simulated_annealing(objective, bounds, n_iterations, step_size, temp):
	# generate an initial point
	best = bounds[:, 0] + rand(len(bounds)) * (bounds[:, 1] - bounds[:, 0])
	# evaluate the initial point
	best_eval = objective(best)
	# current working solution
	curr, curr_eval = best, best_eval
	# run the algorithm
	for i in range(n_iterations):
		# take a step
		candidate = curr + randn(len(bounds)) * step_size
		# evaluate candidate point
		candidate_eval = objective(candidate)
		# check for new best solution
		if candidate_eval < best_eval:
			# store new best point
			best, best_eval = candidate, candidate_eval
			# report progress
			print('>%d f(%s) = %.5f' % (i, best, best_eval))
		# difference between candidate and current point evaluation
		diff = candidate_eval - curr_eval
		# calculate temperature for current epoch
		t = temp / float(i + 1)
		# calculate metropolis acceptance criterion
		metropolis = exp(-diff / t)
		# check if we should keep the new point
		if diff < 0 or rand() < metropolis:
			# store the new current point
			curr, curr_eval = candidate, candidate_eval
	return [best, best_eval]

Poniższy przykład definiuje funkcję, a następnie tworzy wykres liniowy powierzchni odpowiedzi funkcji dla siatki wartości wejściowych i oznacza optima przy f(0,0) = 0,0 z czerwoną linią.

In [ ]:
from numpy import arange
from matplotlib import pyplot

# objective function
def objective(x):
	return x[0]**2.0

# define range for input
r_min, r_max = -5.0, 5.0
# sample input range uniformly at 0.1 increments
inputs = arange(r_min, r_max, 0.1)
# compute targets
results = [objective([x]) for x in inputs]
# create a line plot of input vs result
pyplot.plot(inputs, results)
# define optimal input value
x_optima = 0.0
# draw a vertical line at the optimal input
pyplot.axvline(x=x_optima, ls='--', color='red')
# show the plot
pyplot.show()

Zanim zastosujemy algorytm optymalizacji do problemu, poświęćmy chwilę, aby trochę lepiej zrozumieć kryterium akceptacji. Po pierwsze, harmonogram szybkiego wyżarzania to wykładnicza funkcja liczby iteracji. 

Możemy to wyjaśnić tworząc wykres temperatury dla każdej iteracji algorytmu. Użyjemy temperatury początkowej 10 i 100 iteracji algorytmu.

Uruchomienie przykładu oblicza temperaturę dla każdej iteracji algorytmu i tworzy wykres iteracji algorytmu (oś x) w funkcji temperatury (oś y). 

Widzimy, że temperatura spada szybko - wykładniczo, nie liniowo, po 20 iteracjach jest poniżej 1 i pozostaje niska przez pozostałą część wyszukiwania.

In [ ]:
# explore temperature vs algorithm iteration for simulated annealing
from matplotlib import pyplot
# total iterations of algorithm
iterations = 100
# initial temperature
initial_temp = 10
# array of iterations from 0 to iterations - 1
iterations = [i for i in range(iterations)]
# temperatures for each iterations
temperatures = [initial_temp/float(i + 1) for i in iterations]
# plot iterations vs temperatures
pyplot.plot(iterations, temperatures)
pyplot.xlabel('Iteration')
pyplot.ylabel('Temperature')
pyplot.show()

Przypomnijmy, że kryterium jest funkcją temperatury, ale jest nim również funkcja tego, jak różni się obiektywna ocena nowego punktu w porównaniu z obecnie dzialajacym rozwiązaniem. 

W związku z tym wykreślimy kryterium dla kilku różnych „różnic w celu” wartość funkcji”, aby zobaczyć wpływ, jaki ma to na prawdopodobieństwo akceptacji. 

Uruchomienie przykładu oblicza kryterium akceptacji dla każdej iteracji algorytmu używając temperatury pokazanej dla każdej iteracji. 

Działka ma trzy wiersze dla trzech różnic między nowym, gorszym rozwiązaniem a rozwiązaniem obecnie działającym. Widzimy, że im gorsze rozwiązanie (im większa różnica), tym mniejsze prawdopodobieństwo modelu na zaakceptowanie gorszego rozwiązania, niezależnie od iteracji algorytmu, jak możnaby się spodziewać. 

Możemy zobaczyć też, że we wszystkich przypadkach prawdopodobieństwo zaakceptowania gorszych rozwiązań maleje wraz z iteracją algorytmu.

In [ ]:
# explore metropolis acceptance criterion for simulated annealing
from math import exp
from matplotlib import pyplot
# total iterations of algorithm
iterations = 100
# initial temperature
initial_temp = 10
# array of iterations from 0 to iterations - 1
iterations = [i for i in range(iterations)]
# temperatures for each iterations
temperatures = [initial_temp/float(i + 1) for i in iterations]
# metropolis acceptance criterion
differences = [0.01, 0.1, 1.0]
for d in differences:
	metropolis = [exp(-d/t) for t in temperatures]
	# plot iterations vs metropolis
	label = 'diff=%.2f' % d
	pyplot.plot(iterations, metropolis, label=label)
# inalize plot
pyplot.xlabel('Iteration')
pyplot.ylabel('Metropolis Criterion')
pyplot.legend()
pyplot.show()

Najpierw zainicjujemy generator liczb pseudolosowych. Generalnie nie jest to wymagane, ale w tym przypadku zapewni, że z każdym biegiem otrzymamy te same wyniki (taką samą sekwencję liczb losowych) algorytmu, abyśmy mogli później wykreślić wyniki.

In [ ]:
# objective function
def objective(x):
	return x[0]**2.0

# simulated annealing algorithm
def simulated_annealing(objective, bounds, n_iterations, step_size, temp):
	# generate an initial point
	best = bounds[:, 0] + rand(len(bounds)) * (bounds[:, 1] - bounds[:, 0])
	# evaluate the initial point
	best_eval = objective(best)
	# current working solution
	curr, curr_eval = best, best_eval
	scores = list()
	# run the algorithm
	for i in range(n_iterations):
		# take a step
		candidate = curr + randn(len(bounds)) * step_size
		# evaluate candidate point
		candidate_eval = objective(candidate)
		# check for new best solution
		if candidate_eval < best_eval:
			# store new best point
			best, best_eval = candidate, candidate_eval
			# keep track of scores
			scores.append(best_eval)
			# report progress
			print('>%d f(%s) = %.5f' % (i, best, best_eval))
		# difference between candidate and current point evaluation
		diff = candidate_eval - curr_eval
		# calculate temperature for current epoch
		t = temp / float(i + 1)
		# calculate metropolis acceptance criterion
		metropolis = exp(-diff / t)
		# check if we should keep the new point
		if diff < 0 or rand() < metropolis:
			# store the new current point
			curr, curr_eval = candidate, candidate_eval
	return [best, best_eval, scores]

# seed the pseudorandom number generator
seed(1)
# define range for input
bounds = asarray([[-5.0, 5.0]])
# define the total iterations
n_iterations = 1000
# define the maximum step size
step_size = 0.1
# initial temperature
temp = 10
# perform the simulated annealing search
best, score, scores = simulated_annealing(objective, bounds, n_iterations, step_size, temp)
print('Done!')
print('f(%s) = %f' % (best, score))
# line plot of best scores
pyplot.plot(scores, '.-')
pyplot.xlabel('Improvement Number')
pyplot.ylabel('Evaluation f(x)')
pyplot.show()

Uruchomienie przykładu pokazuje postęp wyszukiwania, w tym numer iteracji, dane wejściowe do funkcji i odpowiedź funkcji celu za każdym razem, gdy następuje wykryta poprawa. Po zakończeniu wyszukiwania zostaje znalezione najlepsze rozwiązanie i raportowana jest jego ocena.

Interesujące może być przejrzenie postępu wyszukiwania w postaci wykresu liniowego, który pokazuje zmianę oceny najlepszego rozwiązania za każdym razem, gdy następuje poprawa. 

Podczas wyszukiwania widzimy około 20 zmian w ocenie funkcji celu za pomocą dużych zmian początkowo i bardzo małych (do niezauważalnych) zmian pod koniec wyszukiwania, algorytm zbiegł się w optimum. 